[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/paddle-converter-colab/blob/main/paddle_converter_colab.ipynb)

In [ ]:
!pip install -q paddlepaddle==2.5.0rc0 paddlenlp==2.5.2 ppdiffusers==0.14.1 
!pip install -q safetensors==0.3.1 omegaconf==2.3.0 fastcore==1.5.29 einops==0.6.1 gradio==3.19.1 numpy==1.24.3 lark==1.1.5 gdown==4.6.4 huggingface-hub==0.13.4

import os, gdown, gc
import gradio as gr
from ppdiffusers import StableDiffusionPipeline
import paddle, torch
from safetensors.torch import save_file, load_file
from huggingface_hub import model_info, create_repo, create_branch, upload_folder
from huggingface_hub.utils import RepositoryNotFoundError, RevisionNotFoundError

def download_ckpt(ckpt_url, is_safetensors):
    if is_safetensors:
        if "drive.google.com" in ckpt_url:
            gdown.download(url=ckpt_url, output="model.safetensors", quiet=False, fuzzy=True)
        else:
            os.system(f"wget {ckpt_url} -O model.safetensors")
        weights = load_file("model.safetensors", device="cpu")
        torch.save(weights, f"model.ckpt")
    else:
        if "drive.google.com" in ckpt_url:
            gdown.download(url=ckpt_url, output="model.ckpt", quiet=False, fuzzy=True)
        else:
            os.system(f"wget {ckpt_url} -O model.ckpt")
    return "download ckpt done!"

def to_paddle():
    os.system("wget -q https://raw.githubusercontent.com/PaddlePaddle/PaddleNLP/develop/ppdiffusers/scripts/convert_diffusers_model/convert_orig_sd_ckpt_to_ppdiffusers.py")
    os.system(f"python3 convert_orig_sd_ckpt_to_ppdiffusers.py --checkpoint_path model.ckpt --dump_path paddle")
    return "convert to paddle done!"

def push_paddle(model_to, token, branch):
    try:
        repo_exists = True
        r_info = model_info(model_to, token=token)
    except RepositoryNotFoundError:
        repo_exists = False
    finally:
        if repo_exists:
            print(r_info)
        else:
            create_repo(model_to, private=True, token=token)
    try:
        branch_exists = True
        b_info = model_info(model_to, revision=branch, token=token)
    except RevisionNotFoundError:
        branch_exists = False
    finally:
        if branch_exists:
            print(b_info)
        else:
            create_branch(model_to, branch=branch, token=token)
    upload_folder(folder_path="paddle", path_in_repo="", revision=branch, repo_id=model_to, commit_message=f"paddle - camenduru/converter", token=token)
    return "push pt done!"
    
def delete_paddle():
    os.system(f"rm -rf paddle")
    return "delete paddle done!"

def download_safetensors(safetensors_url):
    if "drive.google.com" in safetensors_url:
        gdown.download(url=ckpt_url, output="model.safetensors", quiet=False, fuzzy=True)
    else:
        os.system(f"wget {safetensors_url} -O model.safetensors")
    return "download safetensors done!"

def from_safetensors_to_ckpt(ckpt_name):
    weights = load_file("model.safetensors", device="cpu")
    os.system("mkdir ckpt")
    torch.save(weights, f"ckpt/{ckpt_name}.ckpt")
    return "convert to ckpt done!"

def download_vae(vae_url):
    if "drive.google.com" in vae_url:
        gdown.download(url=vae_url, output="vae.ckpt", quiet=False, fuzzy=True)
    else:
        os.system(f"wget {vae_url} -O vae.ckpt")
    return "download vae done!"

def swap_ckpt_vae(ckpt_name):
    os.system("mkdir ckpt")
    model = torch.load("model.ckpt", map_location="cpu")
    if "state_dict" in model:
      sd = model["state_dict"]
    else:
      sd = model
    full_model = False
    vae_model = torch.load("vae.ckpt", map_location="cpu")
    vae_sd = vae_model['state_dict']
    for vae_key in vae_sd:
      if vae_key.startswith("first_stage_model."):
        full_model = True
        break
    for vae_key in vae_sd:
      sd_key = vae_key
      if full_model:
        if not sd_key.startswith("first_stage_model."):
          continue
      else:
        if sd_key not in sd:
          sd_key = "first_stage_model." + sd_key
      if sd_key not in sd:
        continue
      sd[sd_key] = vae_sd[vae_key]
    torch.save(model, f"ckpt/{ckpt_name}-vae-swapped.ckpt")
    del model
    del vae_model
    del sd
    del vae_sd
    gc.collect()
    return "swap ckpt vae done!"

def push_ckpt(model_to, token, branch):
    try:
        repo_exists = True
        r_info = model_info(model_to, token=token)
    except RepositoryNotFoundError:
        repo_exists = False
    finally:
        if repo_exists:
            print(r_info)
        else:
            create_repo(model_to, private=True, token=token)
    try:
        branch_exists = True
        b_info = model_info(model_to, revision=branch, token=token)
    except RevisionNotFoundError:
        branch_exists = False
    finally:
        if branch_exists:
            print(b_info)
        else:
            create_branch(model_to, branch=branch, token=token)    
    upload_folder(folder_path="ckpt", path_in_repo="", revision=branch, repo_id=model_to, commit_message=f"ckpt - camenduru/converter", token=token)
    return "push ckpt done!"
    
def delete_ckpt():
    os.system(f"rm -rf ckpt")
    return "delete ckpt done!"

def delete_all():
    delete_paddle()
    delete_ckpt()
    return "delete all done!"
    
block = gr.Blocks()

with block:
    with gr.Row().style(equal_height=True):
        btn_delete_all = gr.Button("Delete ALL")
        out_all = gr.Textbox(show_label=False)
        btn_delete_all.click(delete_all, outputs=out_all)
    gr.Markdown(
    """
    ### ckpt to ppdiffusers paddle
    ckpt_url = <small>https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt or https://drive.google.com/file/d/file-id/view?usp=share_link or "https://civitai.com/api/download/models/5616?type=Model&format=PickleTensor"</small><br />
    paddle_model_to = camenduru/openjourney <br />
    branch = main <br />
    token = get from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens) new token role=write
    """)
    with gr.Group():
        with gr.Box():
            with gr.Row().style(equal_height=True):
                text_ckpt_url = gr.Textbox(show_label=False, max_lines=1, placeholder="ckpt_url")
                text_paddle_model_to = gr.Textbox(show_label=False, max_lines=1, placeholder="paddle_model_to")
                text_paddle_branch = gr.Textbox(show_label=False, value="main", max_lines=1, placeholder="paddle_branch")
                text_paddle_token = gr.Textbox(show_label=False, max_lines=1, placeholder="🤗 token")
                out_paddle = gr.Textbox(show_label=False)
            with gr.Row().style(equal_height=True):
                is_safetensors = gr.Checkbox(label="Is Safetensors", value=False)
                btn_download_ckpt = gr.Button("Download CKPT")
                btn_to_paddle = gr.Button("Convert to ppDiffusers Paddle")
                btn_push_paddle = gr.Button("Push ppDiffusers Paddle to 🤗")
                btn_delete_paddle = gr.Button("Delete ppDiffusers Paddle")
        btn_download_ckpt.click(download_ckpt, inputs=[text_ckpt_url, is_safetensors], outputs=out_paddle)
        btn_to_paddle.click(to_paddle, outputs=out_paddle)
        btn_push_paddle.click(push_paddle, inputs=[text_paddle_model_to, text_paddle_token, text_paddle_branch], outputs=out_paddle)
        btn_delete_paddle.click(delete_paddle, outputs=out_paddle)
    gr.Markdown(
    """
    ### swap ckpt vae <br />
    ckpt_url = <small>https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt or https://drive.google.com/file/d/file-id/view?usp=share_link or "https://civitai.com/api/download/models/75?type=Model&format=PickleTensor"</small><br />
    vae_url = <small>https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt or https://drive.google.com/file/d/file-id/view?usp=share_link or "https://civitai.com/api/download/models/5809?type=VAE&format=Other"</small><br />
    swaped_ckpt_name = Anything-V3.0 <br />
    swaped_ckpt_model_to = camenduru/Anything-V3.0 <br />
    swaped_ckpt_branch = ckpt <br />
    token = get from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens) new token role=write <br />
    """)
    with gr.Group():
        with gr.Box():
            with gr.Row().style(equal_height=True):
                text_ckpt_url = gr.Textbox(show_label=False, max_lines=1, placeholder="ckpt_url")
                text_vae_url = gr.Textbox(show_label=False, max_lines=1, placeholder="vae_url")
                text_swap_ckpt_name = gr.Textbox(show_label=False, max_lines=1, placeholder="swaped_ckpt_name")
                text_swap_ckpt_model_to = gr.Textbox(show_label=False, max_lines=1, placeholder="swaped_ckpt_model_to")
                text_swap_ckpt_branch = gr.Textbox(show_label=False, value="main", max_lines=1, placeholder="swaped_ckpt_branch")
                text_swap_ckpt_token = gr.Textbox(show_label=False, max_lines=1, placeholder="🤗 token")
                out_swap_ckpt = gr.Textbox(show_label=False)
            with gr.Row().style(equal_height=True):
                btn_download_ckpt = gr.Button("Download CKPT")
                btn_download_vae = gr.Button("Download VAE")
                btn_to_swap_ckpt = gr.Button("Swap CKPT VAE")
                btn_push_swap_ckpt = gr.Button("Push CKPT to 🤗")
                btn_delete_swap_ckpt = gr.Button("Delete CKPT")
        btn_download_ckpt.click(download_ckpt, inputs=[text_ckpt_url], outputs=out_swap_ckpt)
        btn_download_vae.click(download_vae, inputs=[text_vae_url], outputs=out_swap_ckpt)
        btn_to_swap_ckpt.click(swap_ckpt_vae, inputs=[text_swap_ckpt_name], outputs=out_swap_ckpt)
        btn_push_swap_ckpt.click(push_ckpt, inputs=[text_swap_ckpt_model_to, text_swap_ckpt_token, text_swap_ckpt_branch], outputs=out_swap_ckpt)
        btn_delete_swap_ckpt.click(delete_ckpt, outputs=out_swap_ckpt)

block.launch(share=True, inline=False, debug=True)